In [ ]:
import scipy.io as scio
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import *
from sklearn import decomposition
import keras
from keras.utils import *
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling2D, Conv2D, Flatten
from keras.utils import plot_model
%matplotlib inline

#Load the data
data = scio.loadmat("mnist_all.mat")
X_training = []
y_training = []
X_testing = []
y_testing = []
for i in range(10):
    index_string = 'train'+str(i)
    for j in range(len(data[index_string])):
        X_training.append(data[index_string][j]/255.0)
        y_training.append(i)
    index_string = 'test'+str(i)
    for j in range(len(data[index_string])):
        X_testing.append(data[index_string][j]/255.0)
        y_testing.append(i)

#PCA
pca_f = decomposition.PCA(10)
pca_f.fit(X_training)
X_training_pca = pca_f.transform(X_training)
X_testing_pca = pca_f.transform(X_testing)

#NN
y_training_cate = keras.utils.to_categorical(y_training, 10)
y_testing_cate = keras.utils.to_categorical(y_testing, 10)

model = Sequential()
model.add(Dense(40, activation='relu',input_dim=10))
model.add(Dense(40, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
plot_model(model, to_file='model.png')
batch_size = 128
num_epoch = 200
#model training
history = model.fit(np.asarray(X_training_pca), y_training_cate,
          batch_size=batch_size,
          epochs=num_epoch)

# Plot training accuracy values
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

# Plot training loss values
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

score = model.evaluate(np.asarray(X_testing_pca), y_testing_cate)
print('Accuracy:', score[1])